In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Database Reading and Manipulation
import pandas as pd

# Linear Algebra
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Time
import time

# Random and os for reproducibility
import random
import os

# Model Selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

# Modeling
import tensorflow as tf

# Processing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

# Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

# Pipeline
from sklearn.pipeline import Pipeline

# Data imputation
from sklearn.impute import SimpleImputer

# Making keras compatible with scikit learn api
# https://scikit-learn.org/stable/developers/develop.html
from sklearn.base import BaseEstimator, RegressorMixin

# Custom modules
## Model selection
from src.cross_validation.blocking_time_series_split import BlockingTimeSeriesSplit

## Function to print scores
from src.utils.print_scores import print_scores

## Function to calculate score regression metrics
from src.utils.score_regression_metrics import score_regression_metrics

## Function to fill the results metric dict
from src.utils.fill_results_dict import fill_results_dict

# Converting Times Series Data to 3D format
from src.utils.split_sequences import split_sequences

# Transformer Encoder for time series data
from src.models.transformer_ts_tf import Transformer

2023-10-12 14:13:57.094753: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-12 14:13:57.099201: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-12 14:13:57.165968: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-12 14:13:57.167217: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-12 14:13:58.161472: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

<IPython.core.display.Javascript object>

# Functions and definitions

### Convert train/test data to 3D format

In [3]:
def generate_sequences_helper(
    dataset, cement_types, dates=None, timesteps=None, split_by_cement_type=False
):
    index_train = dataset["y_train"].index
    index_test = dataset["y_test"].index

    dataset["y_train"] = dataset["y_train"].reset_index(drop=True)
    dataset["y_test"] = dataset["y_test"].reset_index(drop=True)

    if dates is not None:
        dataset["dates_train"] = dates[index_train].reset_index(drop=True)
        dataset["dates_test"] = dates[index_test].reset_index(drop=True)

    dataset["cement_types_train"] = cement_types.loc[index_train].reset_index(drop=True)
    dataset["cement_types_test"] = cement_types.loc[index_test].reset_index(drop=True)

    dataset = generate_sequences(dataset, timesteps, split_by_cement_type)

    return dataset

<IPython.core.display.Javascript object>

In [4]:
def generate_sequences(dataset, timesteps, split_by_cement_type=False):
    if split_by_cement_type:
        dataset["x_train"], dataset["y_train"] = split_sequences_per_cement_type(
            pd.concat(
                [
                    dataset["dates_train"],
                    pd.DataFrame(dataset["x_train"], columns=x.columns),
                    dataset["cement_types_train"],
                    dataset["y_train"],
                ],
                axis=1,
            ),
            timesteps,
        )

        dataset["x_test"], dataset["y_test"] = split_sequences_per_cement_type(
            pd.concat(
                [
                    dataset["dates_test"],
                    pd.DataFrame(dataset["x_test"], columns=x.columns),
                    dataset["cement_types_test"],
                    dataset["y_test"],
                ],
                axis=1,
            ),
            timesteps,
        )
    else:
        dataset["x_train"], dataset["y_train"] = split_sequences(
            pd.concat(
                [
                    pd.DataFrame(dataset["x_train"], columns=x.columns),
                    dataset["cement_types_train"],
                    dataset["y_train"],
                ],
                axis=1,
            ).values,
            timesteps,
        )

        dataset["x_test"], dataset["y_test"] = split_sequences(
            pd.concat(
                [
                    pd.DataFrame(dataset["x_test"], columns=x.columns),
                    dataset["cement_types_test"],
                    dataset["y_test"],
                ],
                axis=1,
            ).values,
            timesteps,
        )
    return dataset

<IPython.core.display.Javascript object>

### Data preprocessing

In [5]:
def impute_data(dataset, imputer=None, imputer_params=None):
    x_train = dataset["x_train"]
    x_test = dataset["x_test"]

    # Apply imputation to the data
    if imputer is not None:
        imputer = imputer() if imputer_params is None else imputer(**imputer_params)
        x_train = imputer.fit_transform(x_train)
        x_test = imputer.transform(x_test)

    dataset["x_train"] = x_train
    dataset["x_test"] = x_test

    return dataset

<IPython.core.display.Javascript object>

In [6]:
def transform_data(dataset, transformer=None):
    x_train = dataset["x_train"]
    x_test = dataset["x_test"]

    # Apply data normalization/standardization to the data
    if transformer is not None:
        scaler = transformer()
        x_train = scaler.fit_transform(x_train)
        x_test = scaler.transform(x_test)

    dataset["x_train"] = x_train
    dataset["x_test"] = x_test

    return dataset

<IPython.core.display.Javascript object>

In [7]:
def preprocess_data(dataset, transformer=None, imputer=None, imputer_params=None):
    dataset = impute_data(dataset, imputer, imputer_params)
    dataset = transform_data(dataset, transformer)
    return dataset

<IPython.core.display.Javascript object>

### Train and evaluate the model

In [8]:
def train_and_evaluate_model(Estimator, dataset, estimator_params=None):
    """
    Purpose: Helper function to be used in conjunction with
    blocked time_series cross validation function
    """
    x_train = dataset["x_train"]
    y_train = dataset["y_train"]
    x_test = dataset["x_test"]
    y_test = dataset["y_test"]

    # Instantiate the model
    model = Estimator() if estimator_params is None else Estimator(**estimator_params)

    # Fitting the model
    model.fit(x_train, y_train)

    # Making predictions on train/test sets
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    # Return regression metrics
    return score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)

<IPython.core.display.Javascript object>

### Custom Cross Validate

In [9]:
def custom_cross_validate(
    Estimator,
    Imputer,
    Transform,
    x,
    y,
    cv,
    timesteps,
    dates=None,
    cement_types=None,
    estimator_params=None,
    imputer_params=None,
    split_by_cement_type=True,
):
    results = []
    scores = []

    for train_index, test_index in cv.split(x):
        dataset = {
            "dates_train": dates[train_index].reset_index(drop=True),
            "cement_types_train": cement_types.loc[train_index].reset_index(drop=True),
            "x_train": x.loc[train_index].reset_index(drop=True),
            "y_train": y[train_index].reset_index(drop=True),
            "dates_test": dates[test_index].reset_index(drop=True),
            "cement_types_test": cement_types.loc[test_index].reset_index(drop=True),
            "x_test": x.loc[test_index].reset_index(drop=True),
            "y_test": y[test_index].reset_index(drop=True),
        }

        # Preprocess the dataset
        dataset = preprocess_data(dataset, Transform, Imputer, imputer_params)

        # generate sequences (3D format)
        dataset = generate_sequences(dataset, timesteps, split_by_cement_type)

        # Train and Evaluate the model
        score = train_and_evaluate_model(Estimator, dataset, estimator_params)
        scores.append(score)

    # After every iteration metrics results are appended together
    scores_final = {key: [] for key, _ in scores[0].items()}
    for scores_dict in scores:
        for key, value in scores_dict.items():
            scores_final[key] += [value]
    results.append(scores_final)
    return results

<IPython.core.display.Javascript object>

## Model Definition

In [10]:
class TransformerWrapper(RegressorMixin):
    def __init__(self, params):
        self.params = params
        self.model = self.get_model()
        self.batch_size = 16
        self.epochs = 300
        self.verbose = 0

    def fit(self, X=None, y=None):
        self.model.fit(
            X, y, batch_size=self.batch_size, epochs=self.epochs, verbose=self.verbose
        )

    def predict(self, X=None):
        return self.model.predict(X, verbose=self.verbose)

    def get_model(self):
        model = Transformer(
            num_hid=self.params["num_features"],
            time_steps=self.params["timesteps"],
            num_head=self.params["num_heads"],
            num_layers_enc=self.params["num_layers_enc"],
            num_feed_forward=self.params["num_feed_forward"],
        )
        model.compile(
            tf.keras.optimizers.Adam(learning_rate=0.001),
            loss="mse",
            metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
        )
        return model

<IPython.core.display.Javascript object>

In [11]:
def pad_time_series(dataframe, timesteps):
    """
    Pad timeseries with zeros
    """
    df_tmp = pd.DataFrame(
        dict(
            zip(
                dataframe.columns,
                [[0 for _ in range(timesteps - 1)] for _ in range(dataframe.shape[1])],
            )
        )
    )
    df_tmp[DATE] = dataframe[DATE].iloc[0]
    return pd.concat([df_tmp, dataframe], axis=0).reset_index(drop=True)

<IPython.core.display.Javascript object>

In [12]:
def split_sequences_per_cement_type(dataframe, timesteps, pad=False):
    """
    Create sequences per cement time
    to avoid having parts of the sequence
    of different types of cement.
    """
    if timesteps == 1:
        return split_sequences(dataframe.drop(DATE, axis=1).values, timesteps)

    dates = dataframe[DATE][timesteps - 1 :]
    data = []
    dataframes = []

    for cement_type in CEMENT_TYPES:
        data.append(dataframe[dataframe[cement_type] == 1])
    data.append(dataframe[(dataframe[CEMENT_TYPES] == 0).all(axis=1)])
    x, y = split_sequences(dataframe.values, timesteps)

    for df in data:
        if pad:
            dates = df[DATE].reset_index(drop=True)
            df = pad_time_series(df, timesteps).reset_index(drop=True)
        else:
            dates = df[DATE][timesteps - 1 :].reset_index(drop=True)
        x, y = split_sequences(df.drop(DATE, axis=1).values, timesteps)
        x = pd.DataFrame({"Sequences": [sample.tolist() for sample in x]})
        y = pd.DataFrame({"Target": y})
        dataframes.append(pd.concat([dates, x, y], axis=1))

    data = pd.concat(dataframes, axis=0)
    data[DATE] = pd.to_datetime(data[DATE])
    data = data.sort_values(by=DATE).reset_index(drop=True)
    x = data["Sequences"]
    y = data["Target"].values
    x = np.array(x.tolist())
    return x, y

<IPython.core.display.Javascript object>

# Settings for Reproducibility

In [13]:
def set_seeds():
    os.environ["CUDA_VISIBLE_DEVICES"] = ""
    os.environ["PYTHONHASHSEED"] = str(SEED)
    tf.random.set_seed(SEED)
    np.random.seed(SEED)
    random.seed(SEED)


# https://stackoverflow.com/questions/36288235/how-to-get-stable-results-with-tensorflow-setting-random-seed

<IPython.core.display.Javascript object>

In [14]:
def set_global_determinism():
    set_seeds(seed=SEED)

    os.environ["TF_DETERMINISTIC_OPS"] = "1"
    os.environ["TF_CUDNN_DETERMINISTIC"] = "1"

    tf.config.threading.set_inter_op_parallelism_threads(1)
    tf.config.threading.set_intra_op_parallelism_threads(1)

<IPython.core.display.Javascript object>

In [15]:
SEED = 47
METRICS = (
    "neg_root_mean_squared_error",
    "neg_mean_absolute_error",
    "neg_mean_absolute_percentage_error",
    "r2",
)
METRICS_DICT = {
    "neg_root_mean_squared_error": "RMSE",
    "neg_mean_absolute_error": "MAE",
    "neg_mean_absolute_percentage_error": "MAPE",
    "r2": "R2",
}
DATE = "Date"
CEMENT_TYPES = ["cem_type_CEM B", "cem_type_CEM C"]

<IPython.core.display.Javascript object>

## Defining a dataframe structure to save the results

In [16]:
results_to_save = []

results_dict = {
    "Category": "Local Model",
    "Company": "partner_iv",
    "Features": "Chemical + Mineralogical + Physical + One-Hot",
    "Data Shape": None,
    "Timesteps": None,
    "Model": "Transformer",
    "Model Params": None,
    "Scaler": "Standard Scaler",
    "Scaler Params": None,
    "Imputer": "Median",
    "Imputer Params": None,
    "Cross Validation": None,
    "Cross Validation Params": np.nan,
    "RMSE Train": np.nan,
    "MAE Train": np.nan,
    "MAPE Train": np.nan,
    "R2 Train": np.nan,
    "RMSE Test": np.nan,
    "MAE Test": np.nan,
    "MAPE Test": np.nan,
    "R2 Test": np.nan,
}

<IPython.core.display.Javascript object>

# Reading the dataset

In [17]:
df = pd.read_csv("../../../../../data/processed/partner_iv/cement-shipping.csv")

<IPython.core.display.Javascript object>

## Defining Features

In this set of experiments we use all available features

In [18]:
df_copy = df.copy()
df_copy[CEMENT_TYPES] = df_copy[CEMENT_TYPES].astype(int)

<IPython.core.display.Javascript object>

<h2>1. Dataset: df_copy</h2> <br>In this dataset all features are used.

In [19]:
dates = df["Date"].copy()
y = df_copy.pop("CS28")
x = df_copy
df_copy = pd.concat([x, y], axis=1)

<IPython.core.display.Javascript object>

In [20]:
TIMESTEPS_LIST = [1, 7, 14]

<IPython.core.display.Javascript object>

# Transformer Encoder

# Configs:

1. TIMESTEPS: 1, 7, 14

2. Epochs: 300

3. Batch size: 16

4. num heads: 1

5. FF Layers enc: 1

      5.1 Units 16

6. Optimization Algorithm: Adam
    
    6.1. Learning Rate: 1e-3

In [21]:
params = {}
params["timesteps"] = None
params["num_features"] = None
params["num_heads"] = 1
params["num_layers_enc"] = 1
params["num_feed_forward"] = 16

<IPython.core.display.Javascript object>

## 1.1 Repeated KFold Cross validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Timesteps:</b> 1, 3, 5, 7, 10, 15, 20<br>
<b>Repeats:</b>10<br>
<b>Splits:</b>10<br>
    1. 10 folds of 123 samples each
    2. 90% train (1111 samples each fold)
    3. 10% test (123 samples each fold)
<b>Total:</b> 100 models<br>

In [22]:
start = time.time()

repeats = 3
n_splits = 5
TIMESTEPS_LIST = [1, 7, 14]

print("Repeated Cross Validation:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    cv = RepeatedKFold(n_splits=n_splits, n_repeats=repeats, random_state=SEED)
    x = df_copy.drop(["Date", "CS28"] + CEMENT_TYPES, axis=1)
    y = df_copy["CS28"]

    params["timesteps"] = timesteps
    params["num_features"] = x.shape[-1] + len(CEMENT_TYPES)

    scores = custom_cross_validate(
        TransformerWrapper,
        SimpleImputer,
        StandardScaler,
        x,
        y,
        cv,
        timesteps=timesteps,
        dates=dates,
        cement_types=df_copy[CEMENT_TYPES],
        estimator_params={"params": params},
        imputer_params={"strategy": "median"},
        split_by_cement_type=True,
    )
    scores = scores[0]
    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores, METRICS, METRICS_DICT)

    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Repeated KFold"
    results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 3}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(results_dict_copy, scores)
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Repeated Cross Validation:
Repeats: 3
n_splits: 5



2023-10-12 14:14:01.530892: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-10-12 14:14:01.530965: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: inspirada
2023-10-12 14:14:01.530974: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: inspirada
2023-10-12 14:14:01.531190: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 525.125.6
2023-10-12 14:14:01.531239: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 525.125.6
2023-10-12 14:14:01.531250: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 525.125.6


TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 1.296 (0.115)
MAE: 1.041 (0.099)
MAPE: 0.022 (0.002)
R2: 0.961 (0.007)


******
[TEST]
******
RMSE: 2.371 (0.127)
MAE: 1.857 (0.101)
MAPE: 0.038 (0.002)
R2: 0.870 (0.017)


TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.509 (0.171)
MAE: 1.195 (0.163)
MAPE: 0.025 (0.004)
R2: 0.948 (0.012)


******
[TEST]
******
RMSE: 3.738 (0.348)
MAE: 2.926 (0.268)
MAPE: 0.061 (0.006)
R2: 0.675 (0.077)


TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.842 (0.163)
MAE: 1.452 (0.152)
MAPE: 0.030 (0.003)
R2: 0.923 (0.015)


******
[TEST]
******
RMSE: 4.110 (0.512)
MAE: 3.243 (0.366)
MAPE: 0.067 (0.008)
R2: 0.609 (0.106)


Minutes Elapsed:  67.55473660628


<IPython.core.display.Javascript object>

## 1.2. Blocking Time Series Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>5<br>    
    1. 5 folds of 246 samples
    2. 50% train (123 samples each fold)
    3. 50% test (123 samples each fold)
<b>Total:</b> 5 models<br>

In [23]:
start = time.time()

repeats = 3
n_splits = 5
train_size = 0.8
TIMESTEPS_LIST = [1, 7]

print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    scores_final = None

    params["timesteps"] = timesteps

    for _ in range(repeats):
        x = df_copy.drop(["Date", "CS28"] + CEMENT_TYPES, axis=1)
        y = df_copy["CS28"]
        params["num_features"] = x.shape[-1] + len(CEMENT_TYPES)

        cv = BlockingTimeSeriesSplit(n_splits=n_splits, train_size=train_size)

        scores = custom_cross_validate(
            TransformerWrapper,
            SimpleImputer,
            StandardScaler,
            x,
            y,
            cv,
            timesteps,
            dates=dates,
            cement_types=df_copy[CEMENT_TYPES],
            estimator_params={"params": params},
            imputer_params={"strategy": "median"},
            split_by_cement_type=True,
        )
        scores = scores[0]
        if scores_final is None:
            scores_final = {key: [] for key, _ in scores.items()}

        for key, value in scores.items():
            scores_final[key] += [value]

    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores_final, METRICS, METRICS_DICT)

    # Saving the results
    scores = {key: np.array(val).flatten() for key, val in scores_final.items()}
    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Blocking Time Series Split"
    results_dict_copy[
        "Cross Validation Params"
    ] = '{"N_Splits": 5, "Repeats": 3, "train_size": 0.8}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(results_dict_copy, scores)
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Blocking Time Series Split:
Repeats: 3
n_splits: 5

TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 1.458 (0.174)
MAE: 1.199 (0.155)
MAPE: 0.025 (0.003)
R2: 0.949 (0.013)


******
[TEST]
******
RMSE: 3.034 (0.889)
MAE: 2.435 (0.704)
MAPE: 0.050 (0.014)
R2: 0.737 (0.142)


TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.807 (0.145)
MAE: 1.423 (0.125)
MAPE: 0.030 (0.003)
R2: 0.925 (0.014)


******
[TEST]
******
RMSE: 7.804 (4.789)
MAE: 6.108 (3.895)
MAPE: 0.126 (0.079)
R2: -1.145 (2.832)


Minutes Elapsed:  9.415582931041717


<IPython.core.display.Javascript object>

## 1.3. Time Series Split Cross Validation

The training set has size i * n_samples // (n_splits + 1) + n_samples % (n_splits + 1) in the i th split, with a test set of size n_samples//(n_splits + 1) by default, where n_samples is the number of samples.


<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>10<br>    
    1. Train: 10 folds of 114, 226, 338, 450, 562, 675, 787, 899, 1011, 1123 samples each fold
    2. Test: 112 samples each fold
<b>Total:</b> 10 models<br>

In [24]:
set_seeds()
start = time.time()
gap = 0
n_splits = 5
repeats = 3
TIMESTEPS_LIST = [1, 7, 14]

print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    scores_final = None
    params["timesteps"] = timesteps

    for _ in range(repeats):
        x = df_copy.drop(["Date", "CS28"] + CEMENT_TYPES, axis=1)
        y = df_copy["CS28"]
        params["num_features"] = x.shape[-1] + len(CEMENT_TYPES)

        cv = TimeSeriesSplit(
            gap=gap, max_train_size=None, n_splits=n_splits, test_size=None
        )
        scores = custom_cross_validate(
            TransformerWrapper,
            SimpleImputer,
            StandardScaler,
            x,
            y,
            cv,
            timesteps,
            dates=dates,
            cement_types=df_copy[CEMENT_TYPES],
            estimator_params={"params": params},
            imputer_params={"strategy": "median"},
            split_by_cement_type=True,
        )
        scores = scores[0]
        if scores_final is None:
            scores_final = {key: [] for key, _ in scores.items()}

        for key, value in scores.items():
            scores_final[key] += [value]

    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores_final, METRICS, METRICS_DICT)

    # Saving the results
    scores = {key: np.array(val).flatten() for key, val in scores_final.items()}
    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Time Series Split"
    results_dict_copy[
        "Cross Validation Params"
    ] = '{"N_Splits": 5, "Repeats": 3, "Gap": 0}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(results_dict_copy, scores)
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Blocking Time Series Split:
Repeats: 3
n_splits: 5

TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 1.275 (0.187)
MAE: 1.021 (0.172)
MAPE: 0.021 (0.004)
R2: 0.961 (0.012)


******
[TEST]
******
RMSE: 3.163 (0.426)
MAE: 2.478 (0.388)
MAPE: 0.051 (0.010)
R2: 0.754 (0.080)


TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.669 (0.243)
MAE: 1.330 (0.222)
MAPE: 0.028 (0.005)
R2: 0.935 (0.019)


******
[TEST]
******
RMSE: 5.664 (2.090)
MAE: 4.488 (1.603)
MAPE: 0.092 (0.034)
R2: 0.129 (0.727)


TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.917 (0.167)
MAE: 1.505 (0.128)
MAPE: 0.031 (0.003)
R2: 0.916 (0.013)


******
[TEST]
******
RMSE: 6.809 (2.594)
MAE: 5.094 (1.812)
MAPE: 0.105 (0.039)
R2: -0.229 (0.969)


Minutes Elapsed:  47.716764243443805


<IPython.core.display.Javascript object>

## 1.4. Out of time Split Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Train size: 80%</b><br>
<b>Test  size: 20%</b>


<b>Splits:</b> 2<br>    
    1. Train: 987
    2. Test: 247
<b>Total:</b> 1 model<br>

In [25]:
start = time.time()
test_size = 0.2
repeats = 3
TIMESTEPS_LIST = [1, 7, 14]

print("Out of time Cross Val:")
print(f"Repeats: {repeats}")
print(f"Train: {80}%", f"Test: {20}%")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    scores_final = None
    params["timesteps"] = timesteps

    for _ in range(repeats):
        # Data Splitting
        x = df_copy.drop(["Date", "CS28"] + CEMENT_TYPES, axis=1)
        y = df_copy["CS28"]
        params["num_features"] = x.shape[-1] + len(CEMENT_TYPES)

        x_train, x_test, y_train, y_test = train_test_split(
            x, y, test_size=test_size, random_state=SEED, shuffle=False
        )

        # Preprocessing
        imputer = SimpleImputer(strategy="median")
        scaler = StandardScaler()

        x_train = imputer.fit_transform(x_train)
        x_train = scaler.fit_transform(x_train)
        dates_train = dates[: x_train.shape[0]].reset_index(drop=True)
        cement_types_train = df_copy[CEMENT_TYPES][: x_train.shape[0]].reset_index(
            drop=True
        )

        x_test = imputer.transform(x_test)
        x_test = scaler.transform(x_test)
        dates_test = dates[x_train.shape[0] :].reset_index(drop=True)
        cement_types_test = df_copy[CEMENT_TYPES][x_train.shape[0] :].reset_index(
            drop=True
        )

        # Sequence Splitting
        data_train = pd.concat(
            [
                dates_train,
                pd.DataFrame(x_train, columns=x.columns),
                cement_types_train,
                y_train.reset_index(drop=True),
            ],
            axis=1,
        )
        data_test = pd.concat(
            [
                dates_test,
                pd.DataFrame(x_test, columns=x.columns),
                cement_types_test,
                y_test.reset_index(drop=True),
            ],
            axis=1,
        )

        x_train, y_train = split_sequences_per_cement_type(data_train, timesteps)
        x_test, y_test = split_sequences_per_cement_type(data_test, timesteps)

        # Train model and test evalutation
        # Fit model
        pipeline = Pipeline([("estimator", TransformerWrapper(params))])
        pipeline.fit(x_train, y_train)

        # Make predictions
        y_train_pred = pipeline.predict(x_train)
        y_test_pred = pipeline.predict(x_test)

        # evaluate predictions
        scores = score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)

        if scores_final is None:
            scores_final = {key: [] for key, _ in scores.items()}

        for key, value in scores.items():
            scores_final[key] += [value]

    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores_final, METRICS, METRICS_DICT)

    # Saving the results
    # scores = {key: val[0] for key, val in scores.items()}
    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Out of time Split"
    results_dict_copy["Cross Validation Params"] = '{"Test Size": 0.2}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(
        results_dict_copy, {key: value for key, value in scores_final.items()}
    )
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Out of time Cross Val:
Repeats: 3
Train: 80% Test: 20%

TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 1.355 (0.109)
MAE: 1.095 (0.103)
MAPE: 0.023 (0.002)
R2: 0.959 (0.006)


******
[TEST]
******
RMSE: 3.016 (0.075)
MAE: 2.307 (0.010)
MAPE: 0.047 (0.000)
R2: 0.767 (0.012)


TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.554 (0.085)
MAE: 1.206 (0.088)
MAPE: 0.025 (0.002)
R2: 0.946 (0.006)


******
[TEST]
******
RMSE: 4.918 (0.815)
MAE: 4.041 (0.712)
MAPE: 0.087 (0.017)
R2: 0.367 (0.201)


TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 2.091 (0.048)
MAE: 1.708 (0.062)
MAPE: 0.036 (0.001)
R2: 0.904 (0.004)


******
[TEST]
******
RMSE: 4.430 (0.879)
MAE: 3.791 (0.819)
MAPE: 0.082 (0.017)
R2: 0.481 (0.187)


Minutes Elapsed:  15.312968842188518


<IPython.core.display.Javascript object>

In [26]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation", "Cross Validation Params", "Timesteps"],
    dropna=False,
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features        Model  \
                                                                 
0   Chemical + Mineralogical + Physical + One-Hot  Transformer   
1   Chemical + Mineralogical + Physical + One-Hot  Transformer   
2   Chemical + Mineralogical + Physical + One-Hot  Transformer   
3   Chemical + Mineralogical + Physical + One-Hot  Transformer   
4   Chemical + Mineralogical + Physical + One-Hot  Transformer   
5   Chemical + Mineralogical + Physical + One-Hot  Transformer   
6   Chemical + Mineralogical + Physical + One-Hot  Transformer   
7   Chemical + Mineralogical + Physical + One-Hot  Transformer   
8   Chemical + Mineralogical + Physical + One-Hot  Transformer   
9   Chemical + Mineralogical + Physical + One-Hot  Transformer   
10  Chemical + Mineralogical + Physical + One-Hot  Transformer   

              Cross Validation  \
                                 
0   Blocking Time Series Split   
1   Blocking Time Series Split   
2            Out of time Split   
3            Out of time Split   
4            Out of time Split   
5               Repeated KFold   
6               Repeated KFold   
7               Repeated KFold   
8            Time Series Split   
9            Time Series Split   
10           Time Series Split   

                             Cross Validation Params Timesteps RMSE Test  \
                                                                    mean   
0   {"N_Splits": 5, "Repeats": 3, "train_size": 0.8}         1  3.033916   
1   {"N_Splits": 5, "Repeats": 3, "train_size": 0.8}         7  7.804402   
2                                 {"Test Size": 0.2}         1  3.015774   
3                                 {"Test Size": 0.2}         7  4.918220   
4                                 {"Test Size": 0.2}        14  4.429662   
5                      {"N_Splits": 5, "Repeats": 3}         1  2.371174   
6                      {"N_Splits": 5, "Repeats": 3}         7  3.737577   
7                      {"N_Splits": 5, "Repeats": 3}        14  4.110009   
8            {"N_Splits": 5, "Repeats": 3, "Gap": 0}         1  3.163427   
9            {"N_Splits": 5, "Repeats": 3, "Gap": 0}         7  5.664454   
10           {"N_Splits": 5, "Repeats": 3, "Gap": 0}        14  6.809304   

              MAE Test           MAPE Test             R2 Test            
         std      mean       std      mean       std      mean       std  
0   0.889113  2.434862  0.703609  0.049705  0.013607  0.737027  0.141852  
1   4.788841  6.108100  3.895189  0.126344  0.079259 -1.145095  2.831765  
2   0.075197  2.306602  0.009823  0.047198  0.000265  0.767370  0.011674  
3   0.814563  4.040977  0.712030  0.086552  0.016576  0.366737  0.201173  
4   0.879388  3.791149  0.818925  0.081756  0.016903  0.481349  0.186897  
5   0.127168  1.856653  0.100640  0.038289  0.002121  0.869964  0.016643  
6   0.347987  2.925644  0.267542  0.060590  0.005758  0.675123  0.076885  
7   0.511897  3.243300  0.366469  0.067368  0.008246  0.609046  0.105676  
8   0.425514  2.478294  0.387520  0.051016  0.009864  0.754192  0.079540  
9   2.090060  4.488140  1.602943  0.091861  0.034269  0.128723  0.727436  
10  2.594468  5.094182  1.811905  0.104727  0.038862 -0.228724  0.968537

<IPython.core.display.Javascript object>

# Saving the results Dataframe

## Saving the full dataframe

In [27]:
path = "../../../../../reports/results/local_models/partner_iv/all_cements/full/"
filename = "transformer_results_full_1.csv"

pd.concat(results_to_save).to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

## Saving the grouped dataframe

In [28]:
cols_groupby = [
    "Category",
    "Company",
    "Data Shape",
    "Timesteps",
    "Features",
    "Model",
    "Cross Validation",
    "Cross Validation Params",
]

cols_agg = ["RMSE Train", "MAE Train", "MAPE Train", "R2 Train"] + [
    "RMSE Test",
    "MAE Test",
    "MAPE Test",
    "R2 Test",
]

path = "../../../../../reports/results/local_models/partner_iv/all_cements/grouped/"
filename = "transformer_results_grouped_1.csv"


df_results_to_save = (
    pd.concat(results_to_save)
    .groupby(cols_groupby, dropna=False)[cols_agg]
    .agg(["mean", lambda series: pd.Series(series.std(ddof=0), name="std")])
    .reset_index()
    .rename(columns={"<lambda_0>": "std"})
)

df_results_to_save.to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>